In [1]:
%matplotlib widget

In [2]:
import shutil

from core.gain import *
from core.rnn_predic import *
from core.models import *
from core.util import *
#from core.window import WindowGenerator, MissData, make_dataset_water, WaterDataGenerator
from core.window import WindowGenerator, make_dataset_gain, make_dataset_water
from core.file_open import make_dataframe
from core.miss_data import MissData
import json

import os

In [3]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"


In [4]:
data_path = 'data/'
parameters_dir = 'input'

parameters_file = 'input.json'
parameters_path = '{dir}/{file}'.format(dir=parameters_dir, file=parameters_file)

with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

gain_parameters = parameters['gain']
rnn_parameters = parameters['rnn']
file_parameters = parameters['file']

In [5]:
file_parameters

{'watershed': 'han_auto'}

In [6]:
parameters_path = parameters_dir+'/'+ file_parameters['watershed'] + '.json'
with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

In [7]:
pd.set_option('display.max_columns', 1000)

In [8]:
data_parameters = parameters['data']

interpolation_option = data_parameters['interpolation']
colum_idx = data_parameters['columns']
watershed = data_parameters['watershed']
file_names = data_parameters['files']
folder = data_parameters['directorys']
for i in range(len(folder)):
    folder[i] = watershed+folder[i]

In [9]:
__GAIN_TRAINING__ = gain_parameters['train']
gain_epochs = gain_parameters['max_epochs']
gain_in_setps = gain_parameters['input_width']
gain_out_setps = gain_parameters['label_width']
gain_batch_size = gain_parameters['batch_size']
gain_fill_no = gain_parameters['fill_width']
gain_shift = gain_parameters['shift_width']
gain_miss_rate = gain_parameters['miss_rate']

__RNN_TRAINING__ = rnn_parameters['train']
rnn_epochs = rnn_parameters['max_epochs']
rnn_in_setps = rnn_parameters['input_width']
rnn_out_steps = rnn_parameters['label_width']
rnn_batch_size = rnn_parameters['batch_size']
rnn_predict_day = rnn_parameters['predict_day']
rnn_target_column = rnn_parameters['target_column']

if rnn_predict_day < 3 or rnn_predict_day >5:
    print('predict_day err')
    exit(88)
rnn_predict_day -= 1

In [10]:
gain_epochs , rnn_epochs

(2000, 20)

# run file

In [11]:
run_num = range(len(folder))


run_num = [0]




real_df_all = pd.DataFrame([])
target_all = target_mean = target_std = 0

gain_val_performance = {}
gain_performance = {}

length = len(run_num)


In [12]:
for i in range(length):

    idx = run_num[i]

    print('interpol flag : ', interpolation_option[idx])
    print('folder : ', data_path + folder[idx])
    print('colum_idx : ', colum_idx[idx])
    print('file_names[idx] : ', file_names[idx])

    #start = time.time()

    #if watershed == '한강_12days_test':
    #    df, times = make_dataframe_temp_12days(folder[idx], file_names[idx], colum_idx[idx], interpolate=interpolation_option[idx])
    #else:
    df, times = make_dataframe(data_path+folder[idx], file_names[idx], colum_idx[idx], interpolation=interpolation_option[idx])

    df_all, train_mean, train_std, df = normalize(df)



    if i == 0:
        dfff = df
        target_all = df_all
        target_std = train_std
        target_mean = train_mean
        start_year = str(times.iloc[0].year)
        end_year = str(times.iloc[-1].year)

    if interpolation_option[idx][0] == False:

        loadfiles = ['idx.npy', 'miss.npy', 'discriminator.h5', 'generator.h5']

        gain_calc_falg = True

        if __GAIN_TRAINING__ == True:
            gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
            #print(folder[idx], ': training ', 'Miss date save : ', gain_calc_falg)
        else:
            for file in loadfiles:
                if os.path.isfile('save/' + folder[idx]+file):
                    shutil.copyfile('save/' + folder[idx]+file, 'save/'+file)
                    #print('load file name : save/' + folder[idx]+file)
                else:
                    if file == 'miss.npy':
                        gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
                        #print(folder[idx], ': is not miss.npy ', 'Miss date save : ', gain_calc_falg)

        if gain_calc_falg == True:
            #print('GainWindowGenerator in main')
            WindowGenerator.make_dataset = make_dataset_gain
            wide_window = WindowGenerator(input_width=gain_in_setps, label_width=gain_out_setps, shift=gain_shift,
                                          fill_no=gain_fill_no, miss_rate=gain_miss_rate, batch_size=gain_batch_size,
                                          train_df = df_all, val_df = df_all, test_df = df_all, df = df)

            #gain = model_GAIN(shape=wide_window.dg.shape[1:], gen_sigmoid=False, epochs=gain_epochs, training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')
            gain = model_GAIN(shape=(gain_in_setps, df_all.shape[1]), gen_sigmoid=False, epochs=gain_epochs,
                              training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')

            gain_val_performance[str(i)] = gain.evaluate(wide_window.val)
            gain_performance[str(i)] = gain.evaluate(wide_window.test, verbose=0)

            #print('file proc in main')
            if __GAIN_TRAINING__ == True:
                #dir = 'save/'+folder[i]
                if not os.path.exists('save/' + folder[idx]):
                    os.makedirs('save/'+folder[idx])
                for file in loadfiles:
                    shutil.copyfile('save/' + file, 'save/' + folder[idx] + file)

            #print('create_dataset_with_gain in main')
            #ori, gan = create_dataset_with_gain(gain=gain, window=wide_window, df=df)
            ori, gan = create_dataset_with_gain(gain=gain, shape=(gain_in_setps, df_all.shape[1]), df=df)

        else:
            gan = create_dataset_interpol(window=gain_in_setps, df=df)
    else:
        gan = create_dataset_interpol(window=gain_in_setps, df=df)

    if i == 0 :
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.DataFrame(gan)
    else:
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.concat([real_df_all, pd.DataFrame(gan)], axis=1)

interpol flag :  [False, False]
folder :  data/han/자동/
colum_idx :  :,[26,27,28,29,30,31,32,33]
file_names[idx] :  [['가평_2015.xlsx', '가평_2016.xlsx', '가평_2017.xlsx', '가평_2018.xlsx', '가평_2019.xlsx', '가평_2020.xlsx'], ['서상_2015.xlsx', '서상_2016.xlsx', '서상_2017.xlsx', '서상_2018.xlsx', '서상_2019.xlsx', '서상_2020.xlsx'], ['의암호_2015.xlsx', '의암호_2016.xlsx', '의암호_2017.xlsx', '의암호_2018.xlsx', '의암호_2019.xlsx', '의암호_2020.xlsx']]
data/han/자동/가평_2015.xlsx
data/han/자동/가평_2016.xlsx
data/han/자동/가평_2017.xlsx
data/han/자동/가평_2018.xlsx
data/han/자동/가평_2019.xlsx
data/han/자동/가평_2020.xlsx
time range in files :  2015-01-01 00:00  ~  2020-12-31 23:00
data/han/자동/서상_2015.xlsx
data/han/자동/서상_2016.xlsx
data/han/자동/서상_2017.xlsx
data/han/자동/서상_2018.xlsx
data/han/자동/서상_2019.xlsx
data/han/자동/서상_2020.xlsx
time range in files :  2015-01-01 00:00  ~  2020-12-31 23:00
data/han/자동/의암호_2015.xlsx
data/han/자동/의암호_2016.xlsx
data/han/자동/의암호_2017.xlsx
data/han/자동/의암호_2018.xlsx
data/han/자동/의암호_2019.xlsx
data/han/자동/의암호_2020.xlsx
time r

In [13]:
print(real_df_all.shape)

(52608, 36)


In [14]:
def dataset_slice(df, train_ratio, val_ratio, test_ratio):
    total_no = df.shape[0]

    train_no = int(total_no * train_ratio)
    #
    val_no = int(total_no * (train_ratio + val_ratio))

    # val_no = int(total_no*val_ratio)
    # train_no =int(total_no * (train_ratio+val_ratio))
    #
    # val_slice = slice(0, val_no)
    # train_slice = slice(val_no, train_no)
    # test_slice = slice(train_no, None)

    train_slice = slice(0, train_no)    #0.8
    val_slice = slice(train_no, val_no)   #0.1
    test_slice = slice(val_no, None)
    # val_slice = slice(train_no, val_no)   #0.1
    # test_slice = slice(val_no, None)
#     train_slice = slice(0, train_no)  # 0.8
#     val_slice = slice(train_no, None)  # 0.1
#     test_slice = slice(val_no, None)

    train = pd.DataFrame(df[train_slice])
    val = pd.DataFrame(df[val_slice])
    test = pd.DataFrame(df[test_slice])

    test_slice2 = slice(total_no - 288, None)
    test2 = pd.DataFrame(df[test_slice2])

    #print('total_no :1111111111111111111111111 ')
    #print('total_no : ', total_no)
    #print('train : ', train.shape)
    #print('val : ', val.shape)
    #print('test : ', test.shape)


    return train, val, test, test2

In [15]:
train_df, test_df, val_df, test_df2 = dataset_slice(real_df_all, 0.8, 0.1, 0.1)

In [16]:
val_df.shape, test_df.shape, train_df.shape

((5261, 36), (5261, 36), (42086, 36))

In [17]:
print('-------------------prediction')
print('-------------------prediction')
print('-------------------prediction')

print('real_df_all.type : ', type(real_df_all))
print('train_df.type : ', type(train_df))
print('train_df.shape : ', train_df.shape, 'val_df.shape : ', val_df.shape, 'test_df.shape:' ,test_df.shape)


-------------------prediction
-------------------prediction
-------------------prediction
real_df_all.type :  <class 'pandas.core.frame.DataFrame'>
train_df.type :  <class 'pandas.core.frame.DataFrame'>
train_df.shape :  (42086, 36) val_df.shape :  (5261, 36) test_df.shape: (5261, 36)


# target column

In [18]:

label_columns_indices = {name: i for i, name in enumerate(dfff[0])}

print("label_columns_indices:")
print(label_columns_indices)


target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}

rnn_target_column = "chl-a"

print('target columns : ', rnn_target_column)
num_features = dfff[0].shape[1]



target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

print("target_col_idx : ", target_col_idx)
print('out_num_features : ', out_num_features)


label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}
target columns :  chl-a
target_col_idx :  7
out_num_features :  1


In [19]:
val_nse = {}
val_pbias = {}


WindowGenerator.make_dataset = make_dataset_water
multi_window = WindowGenerator(
    input_width=rnn_in_setps,label_width=rnn_out_steps, shift=rnn_out_steps,out_features=out_features,
    out_num_features=out_num_features,label_columns=dfff[0].columns, batch_size=rnn_batch_size,
    train_df=train_df, val_df=val_df, test_df=test_df, test_df2=test_df2)

if __RNN_TRAINING__:
    if not os.path.exists('save/' + watershed):
        os.makedirs('save/' + watershed)


idx = [2, 4, 5, 6, 7]
pa = ["do/", "toc/", "nitrogen/", "phosphorus/", "chlorophyll-a/"]

indices = {name: i for i, name in enumerate(idx)}

model_path = "save/" + watershed + "models/" + pa[indices[target_col_idx]]
print("save model path : ", model_path)

val_nse = {}
val_pbias = {}

 # +"gru.ckpt" -- path




save model path :  save/han/models/chlorophyll-a/


## 모델 학습

In [20]:
rnn_epochs = 7
# multi_linear_model = model_multi_linear(
#     window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
#     #training_flag=__RNN_TRAINING__, checkpoint_path="save/"+watershed+"models/multi_linear.ckpt")
#     training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"multi_linear.ckpt")
# elman_model = model_elman(
#     window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
#     training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"elman.ckpt")
gru_model = model_gru(
    window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
    training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"gru.ckpt")
# multi_lstm_model = model_multi_lstm(
#     window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
#     training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"multi_lstm.ckpt")
# multi_conv_model = model_multi_conv(
#     window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
#     training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"multi_conv.ckpt")


Epoch 1/7
10/10 [==============================] - 11s 855ms/step - loss: 0.6214 - mean_absolute_error: 0.5225 - nse: 0.0845 - val_loss: 0.1654 - val_mean_absolute_error: 0.3370 - val_nse: -0.4112
Epoch 2/7
10/10 [==============================] - 7s 772ms/step - loss: 0.3785 - mean_absolute_error: 0.3785 - nse: 0.4877 - val_loss: 0.1278 - val_mean_absolute_error: 0.2934 - val_nse: -0.0996
Epoch 3/7
10/10 [==============================] - 7s 794ms/step - loss: 0.2692 - mean_absolute_error: 0.3337 - nse: 0.5421 - val_loss: 0.0912 - val_mean_absolute_error: 0.2407 - val_nse: 0.2178
Epoch 4/7
10/10 [==============================] - 8s 868ms/step - loss: 0.2048 - mean_absolute_error: 0.2975 - nse: 0.5840 - val_loss: 0.0970 - val_mean_absolute_error: 0.2546 - val_nse: 0.1625
Epoch 5/7
10/10 [==============================] - 8s 838ms/step - loss: 0.2124 - mean_absolute_error: 0.2967 - nse: 0.6189 - val_loss: 0.0722 - val_mean_absolute_error: 0.2115 - val_nse: 0.3785
Epoch 6/7
10/10 [=====

## core / window.py / 

In [30]:
def hour_to_day_mean(array):
    time = 24
    array = array.reshape((array.shape[0], array.shape[1] // time, time, array.shape[2]))
    array = array.mean(2)
    return array

def compa(model=None,df = None, plot_col=0, input_width=7*24, label_width=5*24, target_std=None, target_mean=None, predict_day=4):
    
    print(df.shape)
    print(plot_col)
    
    width = input_width + label_width
    
    length = df.shape[0]
    length -= width
    
    inputs = []
    labels = []
   
    for i in range(0,length,24):
#         print('i = ', i)
        dataset = df.iloc[i:i+width].to_numpy()
        input = dataset[:input_width]
        label = dataset[input_width:, plot_col:plot_col+1]
        
        input = input.reshape((-1,)+input.shape)
        label = label.reshape((-1,)+label.shape)
        
        inputs.append(input)
        labels.append(label)
        
    inputs = np.concatenate(inputs, axis=0)
    labels = np.concatenate(labels, axis=0)
    
#     print('labels.mean(axis=1)')
#     print(labels.mean(axis=1).shape)
    print(inputs.shape)
    print(labels.shape)

    predictions = model(inputs)
    print(predictions.shape)
    
    predictions = predictions.numpy() * target_std[plot_col] + target_mean[plot_col]
    labels = labels * target_std[plot_col] + target_mean[plot_col]

    print('predictions.shape =', predictions.shape)
#     pred_day = predictions
    pred_day = hour_to_day_mean(predictions)
    print('pred_day.shape =', pred_day.shape)
#     label_day= labels
    label_day = hour_to_day_mean(labels)
    
    inputs_target = inputs[:,:,plot_col:plot_col+1]
    inputs_target = inputs_target * target_std[plot_col] + target_mean[plot_col]
#     inputs_day = inputs_target
    inputs_day = hour_to_day_mean(inputs_target)
    
    #plt.figure(figsize=(10, 800))
    
#     input_index = np.array(range(0,length,24))
#     label_index = input_index + 24*7
    
#     print(input_index)
#     print(label_index)
    
    plt.figure()
    plt.plot(label_day[:, predict_day, :], label='label')
    plt.plot(pred_day[:, predict_day, :], label='pred')
#     plt.plot(input_index, inputs_day[:, 0, :], label='input')
#     plt.plot(label_index, label_day[:, 0, :], label='label')
#     plt.plot(label_index, pred_day[:, 0, :], label='pred')
    plt.legend()
        
        
        
#     o1 = np.mean(labels)
# #     print('labels =', labels)
#     print('o1 =', o1)
#     nse1 = ((labels - predictions)**2).sum()
#     nse2 = ((labels - o1)**2).sum()
#     nse3 = 1 - (nse1/nse2)
    
    
    
#     print('pred_day.shape =',pred_day.shape)
#     print('label_day.shape =',label_day.shape)
    #print('label_index =', label_index)
    
    o1 = np.mean(label_day[:,predict_day,0])
#     o1 = np.mean(label_day)
#     print('labels =', labels)
#     o1 = target_mean[plot_col]
    print('o1 =', o1)
    nse1 = ((pred_day-label_day)**2).sum(axis=0)
    nse2 = ((label_day - o1)**2).sum(axis=0)
    nse3 = 1 - (nse1[predict_day]/nse2[predict_day])
    print('nse3 =', nse3)
    #mean = np.mean(label_day[predict_day])
    print('label_day[predict_day].shape =', label_day[predict_day].shape)
#     print(type(pred_day))
#     print(pred_day[:20, 0, 0])
#     print(label_day[:20, 0, 0])
    o = label_day[:, 0, 0]
    s = pred_day[:, 0, 0]
    mean = np.mean(o)
    print('mean =', mean)
    nse = 1. - np.sum((o-s)**2)/np.sum((o-mean)**2)
#     print('nse =', nse)
    
    
#     pbias1 = (label_day - pred_day).sum(axis=0)
#     pbias2 = (label_day).sum(axis=0)
#     pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    
#     o1 = np.mean(label_day)
# #     print('labels =', labels)
#     print('o1 =', o1)
#     nse1 = ((label_day - pred_day)**2).sum()
#     nse2 = ((label_day - o1)**2).sum()
#     nse3 = 1 - (nse1/nse2)
#     print('nse3 =', nse3)
    #mean = np.mean(label_day[predict_day])
#     print(type(pred_day))
#     print(pred_day[:20, 0, 0])
#     print(label_day[:20, 0, 0])
#     o = label_day[:, 0, 0]
#     s = pred_day[:, 0, 0]
#     mean = np.mean(o)
#     print('mean =', mean)
#     nse = 1. - np.sum((o-s)**2)/np.sum((o-mean)**2)
#     print('nse =', nse)
    
    
    pbias1 = (label_day - pred_day).sum(axis=0)
    pbias2 = (label_day).sum(axis=0)
    pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    
    
    
    return nse3, np.abs(pbias3), pred_day, labels
    

In [31]:
test_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35
42086,0.729297,-1.258776,-0.913277,0.951949,0.058595,0.931013,-0.003343,-0.105533,-0.707105,-1.224741e+00,-1.341300,0.448317,0.694868,2.364266,0.939319,-0.810037,-0.043194,-1.052333,0.284565,-0.756737,-0.707105,-1.224741e+00,-1.341300,0.448317,0.328359,0.266485,0.190977,-0.878869,0.161814,0.916188,-0.063804,0.625522,-0.707105,-1.224741e+00,-1.341300,0.448317
42087,0.741514,-1.225264,-0.889470,0.954522,0.150670,0.869033,-0.003343,-0.101837,-0.999997,-9.999968e-01,-1.340978,0.449279,0.735961,2.495377,1.026215,-0.844775,-0.210419,-1.089896,0.284565,-0.757292,-0.999997,-9.999968e-01,-1.340978,0.449279,0.328359,0.514146,0.261609,-0.856354,0.099473,0.854634,-0.147297,0.576354,-0.999997,-9.999968e-01,-1.340978,0.449279
42088,0.741514,-1.200510,-0.851778,0.879257,0.174925,0.870711,0.116138,-0.107382,-1.224741,-7.071046e-01,-1.340655,0.450239,0.733740,2.509939,1.056769,-0.888519,-0.103802,-1.045446,0.152127,-0.757384,-1.224741,-7.071046e-01,-1.340655,0.450239,0.325027,0.790930,0.334221,-0.884659,0.234475,0.856312,-0.147297,0.536890,-1.224741,-7.071046e-01,-1.340655,0.450239
42089,0.721523,-1.212153,-0.861301,0.919140,0.097362,0.784941,0.068634,-0.188713,-1.366021,-3.660243e-01,-1.340332,0.451200,0.704863,3.092668,1.342851,-0.774655,-0.062592,-1.121825,0.021129,-0.753595,-1.366021,-3.660243e-01,-1.340332,0.451200,0.325027,0.892900,0.328269,-1.040980,0.154716,0.721710,-0.147297,0.410180,-1.366021,-3.660243e-01,-1.340332,0.451200
42090,0.701532,-1.368036,-1.046997,0.877970,0.082821,0.895528,0.165083,-0.290377,-1.414209,-1.274208e-11,-1.340008,0.452160,0.660439,3.165496,1.232543,-0.743134,-0.222546,-1.148745,-0.003343,-0.753779,-1.414209,-1.274208e-11,-1.340008,0.452160,0.320585,0.441300,0.215185,-1.051916,0.207883,0.568326,-0.147297,0.384301,-1.414209,-1.274208e-11,-1.340008,0.452160
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47342,0.572127,0.159156,-0.152282,0.660895,0.401710,0.146538,0.826560,-0.422916,-0.707105,-1.224741e+00,0.824224,-1.149473,0.286154,0.047966,0.294939,0.414797,-1.027389,-0.688594,0.694229,-0.515794,-0.707105,-1.224741e+00,0.824224,-1.149473,0.557857,1.387556,0.134024,0.744115,0.752311,-1.295579,0.252637,0.040396,-0.707105,-1.224741e+00,0.824224,-1.149473
47343,0.419013,0.066588,-0.278988,0.521789,0.249140,0.081783,0.679375,-0.213184,-0.999997,-9.999968e-01,0.823400,-1.150063,0.320585,0.179078,0.436592,0.344035,-1.691216,-0.688594,0.140611,-0.546016,-0.999997,-9.999968e-01,0.823400,-1.150063,0.653775,1.752413,0.083055,0.734515,0.654974,-1.239509,0.198150,0.061731,-0.999997,-9.999968e-01,0.823400,-1.150063
47344,0.407000,-0.014988,-0.269768,0.503992,0.357030,0.106663,0.673603,-0.262967,-1.224741,-7.071046e-01,0.822575,-1.150653,0.329469,0.499566,0.562770,0.299004,-1.669404,-0.755382,0.668922,-0.511358,-1.224741,-7.071046e-01,0.822575,-1.150653,0.644889,1.665006,0.066785,0.723579,0.712631,-1.322882,0.282920,0.270859,-1.224741,-7.071046e-01,0.822575,-1.150653
47345,0.405096,0.126349,-0.127923,0.454462,0.440694,-0.164778,0.407683,-0.088335,-1.366021,-3.660243e-01,0.821750,-1.151242,0.320585,0.747227,0.615541,0.228241,-1.548217,-0.603876,0.860380,-0.465978,-1.366021,-3.660243e-01,0.821750,-1.151242,0.648221,1.825259,0.103686,0.723579,1.108795,-1.286478,0.263849,0.144092,-1.366021,-3.660243e-01,0.821750,-1.151242


In [32]:
#val_nse['Linear'], val_pbias['Linear'], pred, label = compa(
#    model=gru_model,df=test_df,
#    plot_col=out_features[0], target_std=target_std, target_mean=target_mean, predict_day = rnn_predict_day)

val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    model=gru_model,df=test_df,
    plot_col=out_features[0], target_std=target_std, target_mean=target_mean, predict_day = 4)


print()
print()



print(val_nse['GRU'])


print(val_pbias['GRU'])
#print(pred)
#print(label)

(5261, 36)
7
(208, 168, 36)
(208, 120, 1)
(208, 120, 1)
predictions.shape = (208, 120, 1)
pred_day.shape = (208, 5, 1)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

o1 = 14.970594
nse3 = [0.6118906]
label_day[predict_day].shape = (5, 1)
mean = 14.875405


[0.6118906]
[17.617182]


In [33]:
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    model=gru_model,df=train_df,
    plot_col=out_features[0], target_std=target_std, target_mean=target_mean, predict_day = 4)

print()
print()

print(val_nse['GRU'])
print(val_pbias['GRU'])

(42086, 36)
7
(1742, 168, 36)
(1742, 120, 1)
(1742, 120, 1)
predictions.shape = (1742, 120, 1)
pred_day.shape = (1742, 5, 1)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

o1 = 9.062026
nse3 = [0.6658702]
label_day[predict_day].shape = (5, 1)
mean = 9.057987


[0.6658702]
[4.153232]


In [34]:
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    model=gru_model,df=real_df_all,
    plot_col=out_features[0], target_std=target_std, target_mean=target_mean, predict_day = 4)

print()
print()

print(val_nse['GRU'])
print(val_pbias['GRU'])

(52608, 36)
7
(2180, 168, 36)
(2180, 120, 1)
(2180, 120, 1)
predictions.shape = (2180, 120, 1)
pred_day.shape = (2180, 5, 1)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

o1 = 9.570749
nse3 = [0.6779874]
label_day[predict_day].shape = (5, 1)
mean = 9.5582695


[0.6779874]
[6.4641833]
